In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    ManagedOnlineEndpoint,
    Model,
    CodeConfiguration,
    Environment,
)

import random

In [2]:
# Create handle to ml workspace
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: /config.json
Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [3]:
rand = random.randint(0, 10000)
endpoint_name = f"endpt-moe-{rand}"
print(endpoint_name)

endpt-moe-9216


In [4]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key")

In [ ]:
# create endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
# print endpoint identity details
## endpoint's identity is already given blob reader permission on the default storage account
endpoint = ml_client.online_endpoints.get(endpoint_name)
print(endpoint.identity.type)
print(endpoint.identity.principal_id)

## By default Managed Endpoint identity is assigned ACR Pull role on ACR and Blob Reader role on Storage account.
## Remember to assign blob contributor role on storage account for this ID (manual step)

system_assigned
60fd5b38-f8d0-48d6-a242-6366952f91db


In [7]:
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=Model(path="./model/"),
    code_configuration=CodeConfiguration(
        code="./onlinescoring/", scoring_script="score_sai.py"
    ),
    environment=Environment(
        conda_file="./environment/conda-managedidentity.yml",
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
    environment_variables={
        "SAI_APP_ID": endpoint.identity.principal_id,
    },
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

In [ ]:
# set traffic to 100% for deployment
endpoint.traffic = {str(deployment.name): 100}
ml_client.begin_create_or_update(endpoint).result()

In [10]:
# test endpoint
sample_data = "./invoke/sample-request.json"
ml_client.online_endpoints.invoke(endpoint_name=endpoint_name, request_file=sample_data)

'[11055.977245525679, 4503.079536107787]'